<a href="https://colab.research.google.com/github/bobuallah/CG/blob/main/CG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ☁️ **OmniStream™ Cloud Console**
> *The Infinite, Zero-Latency Cloud Gaming Rig for the Underground.*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/)
![Status](https://img.shields.io/badge/Status-Operational-brightgreen)
![Platform](https://img.shields.io/badge/Platform-Google_Colab_T4-blue)
![License](https://img.shields.io/badge/License-MIT-orange)

---

## 🖥️ **Project Overview**
**OmniStream** is a specialized "Set-and-Forget" cloud gaming architecture designed to bypass hardware limitations. It leverages Google's high-speed datacenter infrastructure to create a persistent, high-performance Windows gaming environment inside a Linux container.

### **✨ Key Features**
* **🚀 Zero-Touch Launch:** One click installs Steam, Epic (Heroic), and Drivers automatically.
* **🎮 Native Controller Support:** Powered by Steam Link's backend for 0ms input lag.
* **💾 Infinite Persistence:** Automatically backs up Game Saves & Logins to Google Drive every 10 minutes.
* **⚡ Gigabit Flash-Load:** Downloads 100GB+ games in minutes using Datacenter Bandwidth.

---

## 🛠️ **How to Use**
1.  **Run the Cell Below:** Click the "Play" button on the **OmniStream Engine** code block.
2.  **Wait for Initialization:** The system will install dependencies (~4 mins).
3.  **Connect:** Click the `trycloudflare.com` link that appears in the logs.
4.  **Play:**
    * **Login:** Sign into Steam/Epic inside the VNC window (First run only).
    * **Config:** Set Controller to "Desktop Mode" in Steam Settings.
    * **Game:** Launch your game via Steam.
5.  **Forget:** The system auto-saves your progress. Just close the tab when done.

---

## ⚠️ **System Requirements**
* **Client:** Android Phone with **Steam Link App** or PC/Browser.
* **Input:** Xbox, PS or 3rd party Controllers (via Bluetooth/USB).
* **Account:** A Google Account with Drive access (for save backups).

> *Built by @Moustuofa | Version 3.0 (Elite Edition)*

In [ ]:
# @title 🚀 **OmniStream Engine** (Click Run & Game)
# @markdown This script initializes the persistent cloud rig, restores your data, and manages auto-backups.

import os, subprocess, time, datetime, sys

# ==========================================
# ⚙️ CONFIGURATION
# ==========================================
# Auto-save interval in seconds (600 = 10 minutes)
SAVE_INTERVAL = 600
DRIVE_ROOT = "/content/drive/MyDrive/OmniStream_Data"
CONFIG_BACKUP = f"{DRIVE_ROOT}/Configs"
SAVE_BACKUP = f"{DRIVE_ROOT}/Saves"

# ==========================================
# 1. INITIALIZATION & RESTORE
# ==========================================
from google.colab import drive
if not os.path.exists('/content/drive'):
    print("🔌 Connecting to OmniStream Drive Vault...")
    drive.mount('/content/drive')

os.makedirs(CONFIG_BACKUP, exist_ok=True)
os.makedirs(SAVE_BACKUP, exist_ok=True)

print("🛠️  Building System Architecture (Approx 4 mins)...")
# Quietly install dependencies
subprocess.run("dpkg --add-architecture i386 && apt-get update", shell=True)
pkgs = "steam xfce4 xfce4-terminal python3-pip xdotool libfuse2 aria2 dbus-x11"
subprocess.run(f"apt-get install -y {pkgs} > /dev/null 2>&1", shell=True)

# Install Heroic Launcher (Epic/GOG)
if not os.path.exists("/usr/bin/heroic"):
    print("⚔️  Installing Heroic Launcher...")
    heroic_url = "https://github.com/Heroic-Games-Launcher/HeroicGamesLauncher/releases/download/v2.10.0/heroic_2.10.0_amd64.deb"
    subprocess.run(f"wget -q -O heroic.deb {heroic_url} && apt-get install -y ./heroic.deb > /dev/null 2>&1", shell=True)

# Install Ludusavi (The Auto-Saver)
if not os.path.exists("/usr/bin/ludusavi"):
    print("💾  Installing Ludusavi Persistence Layer...")
    ludu_url = "https://github.com/mtkennerly/ludusavi/releases/download/v0.21.0/ludusavi-v0.21.0-linux.tar.gz"
    subprocess.run(f"wget -q -O ludusavi.tar.gz {ludu_url} && tar -xzf ludusavi.tar.gz && mv ludusavi /usr/bin/ && chmod +x /usr/bin/ludusavi", shell=True)

print("🔄  Restoring User Data & Configs...")
# Restore Steam
if os.path.exists(f"{CONFIG_BACKUP}/Steam_Config.tar.gz"):
    os.makedirs("/root/.local/share/Steam", exist_ok=True)
    subprocess.run(f"tar -xzf '{CONFIG_BACKUP}/Steam_Config.tar.gz' -C /", shell=True)

# Restore Heroic
if os.path.exists(f"{CONFIG_BACKUP}/Heroic_Config.tar.gz"):
    os.makedirs("/root/.config/heroic", exist_ok=True)
    subprocess.run(f"tar -xzf '{CONFIG_BACKUP}/Heroic_Config.tar.gz' -C /", shell=True)

# Restore Game Saves
if os.path.exists(f"{SAVE_BACKUP}/Latest_Saves.zip"):
    subprocess.run(f"ludusavi restore --force --path '{SAVE_BACKUP}/Latest_Saves.zip'", shell=True)

# ==========================================
# 2. LAUNCH SERVICES
# ==========================================
print("🚀  Initializing VNC & Steam Big Picture...")
subprocess.Popen(["vncserver", ":1", "-geometry", "1280x720", "-depth", "24"])
os.environ["DISPLAY"] = ":1"
subprocess.Popen(["steam", "-bigpicture", "-no-browser", "-silent"])

# Cloudflare Tunnel
if not os.path.exists("/usr/bin/cloudflared"):
    subprocess.run("wget -q -O cloudflared-linux-amd64 https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", shell=True)
    subprocess.run("chmod +x cloudflared-linux-amd64 && mv cloudflared-linux-amd64 /usr/bin/cloudflared", shell=True)

# Start Tunnel
tunnel_process = subprocess.Popen(["cloudflared", "tunnel", "--url", "vnc://localhost:5901"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# ==========================================
# 3. INFINITE LOOP (MONITOR & BACKUP)
# ==========================================
print(f"✅  OMNISTREAM ONLINE. Auto-backup active ({SAVE_INTERVAL}s).")
print("👇  CONNECT USING THE LINK BELOW  👇")

last_backup = time.time()

while True:
    # 1. Print Tunnel Output
    line = tunnel_process.stderr.readline()
    if line:
        decoded_line = line.decode('utf-8').strip()
        if "trycloudflare.com" in decoded_line:
            print(f"\n🔗  ACCESS LINK: {decoded_line}\n")

    # 2. Auto-Backup Check
    if time.time() - last_backup > SAVE_INTERVAL:
        print(f"💾  [System] Performing Auto-Backup at {datetime.datetime.now().strftime('%H:%M')}...")

        # Backup Saves
        subprocess.run(f"ludusavi backup --force --path '{SAVE_BACKUP}/Latest_Saves.zip'", shell=True)

        # Backup Steam Login
        steam_paths = ["/root/.local/share/Steam/config", "/root/.local/share/Steam/userdata"]
        tar_cmd = f"tar -czf '{CONFIG_BACKUP}/Steam_Config.tar.gz' " + " ".join(steam_paths)
        subprocess.run(tar_cmd, shell=True)

        # Backup Heroic Login
        if os.path.exists("/root/.config/heroic"):
            subprocess.run(f"tar -czf '{CONFIG_BACKUP}/Heroic_Config.tar.gz' '/root/.config/heroic'", shell=True)

        print("✅  [System] Backup Secured.")
        last_backup = time.time()

    time.sleep(0.5)

🔌 Connecting to OmniStream Drive Vault...
